In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *

apply_storage_config(spark)

In [0]:
dbutils.fs.ls(silver_path)

In [0]:
path_storage_silver = f"{silver_path}"
path_storage_gold = f"{gold_path}"

In [0]:
from pyspark.sql.functions import (
    col,                    
    when,                   
    lit,                    
    trim,                   
    upper,                 
    lower,                         
    coalesce,              
    concat,                            
    substring,             
    length,                
    cast,                  
    round,                
    sum,                   
    count,                 
    avg,                   
    max,                   
    min,                   
    countDistinct,         
    current_timestamp,     
    year,                  
    month,                 
    datediff,              
    to_date,               
    date_format,           
    rank,                  
    dense_rank,            
    lag,                  
    lead                   
)

from pyspark.sql.types import (
    StringType,
    IntegerType,
    DoubleType,
    DateType,
    TimestampType,
    DecimalType
)


In [0]:
# ler arquivos delta da bronze 'exp - imp'
df_exp_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/exp_tratado")

df_imp_tratado = spark.read.format("delta").load(f"{path_storage_silver}/balancacomercial/imp_tratado")


#  Ler Tabelas para (Dimensao TEMPO)

- Colunas de referencia 
- CO_ANO
- CO_ANO

In [0]:
df_dim_tempo = df_imp_tratado  


datas_exp = df_exp_tratado.select("CO_ANO", "CO_MES")
datas_imp = df_imp_tratado.select("CO_ANO", "CO_MES")

df_datas_base = datas_exp.union(datas_imp).distinct()

In [0]:
from pyspark.sql import functions as F

dim_tempo = df_datas_base.withColumn("CO_TRIMESTRE", F.ceil(F.col("CO_MES") / 3)) \
                         .withColumn("CO_SEMESTRE", F.ceil(F.col("CO_MES") / 6)) \
                         .withColumn("NO_TRIMESTRE", F.concat(F.col("CO_TRIMESTRE"), F.lit("º Trimestre"))) \
                         .withColumn("NO_SEMESTRE", F.concat(F.col("CO_SEMESTRE"), F.lit("º Semestre"))) \
                         .orderBy("CO_ANO", "CO_MES")

display(dim_tempo)

In [0]:
from pyspark.sql.functions import sha2, concat_ws, col
dim_tempo = (

    dim_tempo
    .withColumn(
        "sk_tempo",
        sha2(concat_ws("||", col("CO_ANO"), col("CO_MES")), 256)
    )
)

display(dim_tempo)

SALVAR DIMENSÃO NA GOLD

In [0]:
  # Salvar na Gold
dim_tempo.write \
 \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "True") \
    .save(f"{gold_path}dim_tempo/")

In [0]:
# salvar na Gold metastore

dim_tempo.write \
    .mode("overwrite") \
    .saveAsTable("gold.dim_tempo")

In [0]:
%sql
select * from gold.dim_tempo limit 10;

### Ler Tabelas para (Dimensao produto)

- Tabelas de referencia 
-  NCM + NCM_SH + NCM_ISIC + NCM_CGCE + NCM_FAT_AGREG

In [0]:
# Necessário adaptar para SQL e puxar das tabelas silver
df_dim_produto = ncm_tratado

In [0]:
# Necessário adaptar para SQL e puxar das tabelas silver
df_dim_produto = df_dim_produto.join(ncm_sh_tratado, on='CO_SH6', how='left') \
                    .join(ncm_isic_tratado, on='CO_ISIC_CLASSE', how='left') \
                    .join(ncm_cgce_tratado, on='CO_CGCE_N3', how='left') \
                    .join(ncm_fat_agreg_tratado, on='CO_FAT_AGREG', how='left')


In [0]:

# Necessário adaptar para SQL e puxar das tabelas silver
dim_produto = (
    df_dim_produto
    .withColumn(
        "sk_produto",
        sha2(concat_ws("||", col("CO_NCM")), 256)
    )
)

In [0]:
# Salvar na Silver
dim_produto.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "True") \
    .save(f"{path_storage_gold}dim_produto/")
    

In [0]:
dim_produto.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("gold.dim_produtos")

In [0]:
%sql
SELECT * FROM gold.dim_produtos;

### DIMENSÃO - DIM_LOCALIDADE_BR (UF + UF_MUN)

In [0]:

# Necessário adaptar para SQL e puxar das tabelas silver
df_dim_localidade_br = uf_mun_tratado.join(
        uf_tratado,
        uf_mun_tratado.SG_UF == uf_tratado.SG_UF,
        "left"
    ).select(
        uf_tratado.CO_UF,
        uf_mun_tratado.CO_MUN_GEO,
        uf_mun_tratado.NO_MUN_MIN,
        uf_mun_tratado.SG_UF,
        uf_tratado.NO_UF,
        uf_tratado.NO_REGIAO
    )

In [0]:
# Necessário adaptar para SQL e puxar das tabelas silver
dim_localidade_br = df_dim_localidade_br.withColumn(
    "sk_localidade",
    sha2(concat_ws("||", col("NO_UF"), col("CO_MUN_GEO")), 256)
)

In [0]:
# Salvar na Silver
dim_localidade_br.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "True") \
    .save(f"{path_storage_gold}dim_localidade_br/")

In [0]:
dim_localidade_br.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("gold.dim_localidades_br")

In [0]:
%sql

SELECT * FROM gold.dim_localidades_br

### DIM_GEOGRAFIA: PAIS + PAIS_BLOCO.



In [0]:
# Necessário adaptar para SQL e puxar das tabelas silver
df_dim_geografia = df_pais_filtrado

In [0]:
# Necessário adaptar para SQL e puxar das tabelas silver
df_dim_geografia = df_pais_filtrado.join(df_pais_bloco_filtrado, on='CO_PAIS', how='left') 
display(df_dim_geografia)

In [0]:
# Necessário adaptar para SQL e puxar das tabelas silver
# alterar valores nulos por 'Não Definido'
df_dim_geografia = df_dim_geografia.fillna('Não Definido', subset=['NO_BLOCO'])
df_dim_geografia = df_dim_geografia.fillna('Não Definido', subset=['CO_BLOCO'])

df_dim_geografia.display()

In [0]:
# Necessário adaptar para SQL e puxar das tabelas silver
dim_geografia = (
    df_dim_geografia
    .withColumn(
        "sk_geografia",
        sha2(concat_ws("||", col("CO_PAIS")), 256)
    )
)

In [0]:
# Salvar na Silver
dim_geografia.write \
 \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "True") \
    .save(f"{path_storage_gold}dim_geografia/")

In [0]:
# gravar tabela na camada Silver
dim_geografia.write \
    .mode("overwrite") \
    .saveAsTable("gold.dim_geografia")

In [0]:
%sql
select * from gold.dim_geografia limit 10;